# Minutes tutorial

In [1]:
# setup
from datetime import datetime

import pandas as pd
import exchange_calendars as xcals

### `minute` parameter
Methods that require a single minute to be specified take a `minute` parameter. Those that act on a range of minutes take `start` and `end` parameters.

A `minute` parameter can take a `Minute` or a `TradingMinute` type, defined as:

```python
Minute = typing.Union[pd.Timestamp, str, int, float, datetime.datetime]
TradingMinute = Minute
```
In short, a `minute` parameter can take any type that can be passed as a single argument to pd.Timestamp(). For example, the argument of `next_minute` takes a `Minute` type, such any of the following inputs are valid:

In [2]:
inputs = [
    "2021-06-15 14:33",
    pd.Timestamp("2021-06-15 14:33"),
    datetime(2021, 6, 15, 14, 33),
    1623767580000000000,
]
lon = xcals.get_calendar("XLON")
for input in inputs:
    assert lon.next_minute(input) == pd.Timestamp('2021-06-15 14:34', tz='UTC')

The difference between `Minute` and `TradingMinute` is that whilst an object passed to a parameter annotated `Minute` can represent any minute, an object passed to a parameter annotated `TradingMinute` must represent a 'trading minute', i.e. a minute when the exchange is open. To the contrary the method will raise a `NotTradingMinuteError`.

For example, the first argument of `minutes_window` takes a `TradingMinute`, such that whilst this is valid:

In [3]:
lon.minutes_window(inputs[0], count=3)

DatetimeIndex(['2021-06-15 14:33:00+00:00', '2021-06-15 14:34:00+00:00',
               '2021-06-15 14:35:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

but this isn't...

In [ ]:
lon.minutes_window("2021-06-15 22:30", count=3)

```
---------------------------------------------------------------------------
NotTradingMinuteError                     Traceback (most recent call last)
Input In [4], in <cell line: 1>()
----> 1 lon.minutes_window("2021-06-15 22:30", count=3)

NotTradingMinuteError: Parameter `minute` takes a trading minute although received input that parsed to '2021-06-15 22:30:00+00:00' which is not a trading minute of calendar 'XLON'.
```

To find out **which type a `minute` parameter takes**, simply refer to the annotated types in the method signature:

In [ ]:
lon.minutes_window?

```
Signature:
lon.minutes_window(
    minute: 'TradingMinute',
    count: 'int',
    _parse: 'bool' = True,
) -> 'pd.DatetimeIndex'
...
```

### Which minutes are treated as trading minutes?

Any minute that represents a time when an exchange is open is referred to as a 'trading minute'. At a session's bounds, which of a session's open/close and break start/end are considered as trading minutes is determined by the calendar's `side` parameter:

* **"left"** - treat session open and break-end as trading minutes,
    do not treat session close or break-start as trading minutes.
* **"right"** - treat session close and break-start as trading minutes,
    do not treat session open or break-end as tradng minutes.
* **"both"** - treat all of session open, session close, break-start
    and break-end as trading minutes.
* **"neither"** - treat none of session open, session close,
    break-start or break-end as trading minutes.

So, looking at the session "2021-06-15" for XHKG (Hong Kong):

In [6]:
hkg_left = xcals.get_calendar("XHKG")
hkg_left.schedule.loc[["2021-06-15"]]

,open,break_start,break_end,close
2021-06-15,2021-06-15 01:30:00+00:00,2021-06-15 04:00:00+00:00,2021-06-15 05:00:00+00:00,2021-06-15 08:00:00+00:00


**NB** open, close, and break times are **independent of a calendar's `side`**. These properties reflect times corresponding with the start and end of regular trading periods.

In [7]:
# the default side is "left", such that the open is a trading minute...
hkg_left.is_trading_minute("2021-06-15 01:30")

True

In [8]:
# ...and so is the break_end (i.e. the open of the afternoon subsession)...
hkg_left.is_trading_minute("2021-06-15 05:00")

True

In [9]:
# but the close is not a trading minute
hkg_left.is_trading_minute("2021-06-15 08:00")

False

In [10]:
# and neither is the break_start (i.e. the close of the morning subsession)...
hkg_left.is_trading_minute("2021-06-15 04:00")

False

In [11]:
# everything's reversed when `side` is "right"...
hkg_right = xcals.get_calendar("XHKG", side="right")
times = ["01:30", "05:00", "08:00", "04:00"]
[ hkg_right.is_trading_minute("2021-06-15 " + tm) for tm in times ]

[False, False, True, True]

In [12]:
# everything's True when `side` is "both"...
hkg_both = xcals.get_calendar("XHKG", side="both")
[ hkg_both.is_trading_minute("2021-06-15 " + tm) for tm in times ]

[True, True, True, True]

In [13]:
# and False when `side` is "neither".
hkg_neither = xcals.get_calendar("XHKG", side="neither")
[ hkg_neither.is_trading_minute("2021-06-15 " + tm) for tm in times ]

[False, False, False, False]

The effect of the calendar's side can be seen in all methods that interrogate or evaluate trading minutes. For example, note the effect of different sides on the minutes associated with the XHKG session "2021-06-15":

In [14]:
session = "2021-06-15"
hkg_left.session_minutes(session)

DatetimeIndex(['2021-06-15 01:30:00+00:00', '2021-06-15 01:31:00+00:00',
               '2021-06-15 01:32:00+00:00', '2021-06-15 01:33:00+00:00',
               '2021-06-15 01:34:00+00:00', '2021-06-15 01:35:00+00:00',
               '2021-06-15 01:36:00+00:00', '2021-06-15 01:37:00+00:00',
               '2021-06-15 01:38:00+00:00', '2021-06-15 01:39:00+00:00',
               ...
               '2021-06-15 07:50:00+00:00', '2021-06-15 07:51:00+00:00',
               '2021-06-15 07:52:00+00:00', '2021-06-15 07:53:00+00:00',
               '2021-06-15 07:54:00+00:00', '2021-06-15 07:55:00+00:00',
               '2021-06-15 07:56:00+00:00', '2021-06-15 07:57:00+00:00',
               '2021-06-15 07:58:00+00:00', '2021-06-15 07:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=330, freq=None)

In [15]:
hkg_right.session_minutes(session)

DatetimeIndex(['2021-06-15 01:31:00+00:00', '2021-06-15 01:32:00+00:00',
               '2021-06-15 01:33:00+00:00', '2021-06-15 01:34:00+00:00',
               '2021-06-15 01:35:00+00:00', '2021-06-15 01:36:00+00:00',
               '2021-06-15 01:37:00+00:00', '2021-06-15 01:38:00+00:00',
               '2021-06-15 01:39:00+00:00', '2021-06-15 01:40:00+00:00',
               ...
               '2021-06-15 07:51:00+00:00', '2021-06-15 07:52:00+00:00',
               '2021-06-15 07:53:00+00:00', '2021-06-15 07:54:00+00:00',
               '2021-06-15 07:55:00+00:00', '2021-06-15 07:56:00+00:00',
               '2021-06-15 07:57:00+00:00', '2021-06-15 07:58:00+00:00',
               '2021-06-15 07:59:00+00:00', '2021-06-15 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=330, freq=None)

In [16]:
hkg_both.session_minutes(session)

DatetimeIndex(['2021-06-15 01:30:00+00:00', '2021-06-15 01:31:00+00:00',
               '2021-06-15 01:32:00+00:00', '2021-06-15 01:33:00+00:00',
               '2021-06-15 01:34:00+00:00', '2021-06-15 01:35:00+00:00',
               '2021-06-15 01:36:00+00:00', '2021-06-15 01:37:00+00:00',
               '2021-06-15 01:38:00+00:00', '2021-06-15 01:39:00+00:00',
               ...
               '2021-06-15 07:51:00+00:00', '2021-06-15 07:52:00+00:00',
               '2021-06-15 07:53:00+00:00', '2021-06-15 07:54:00+00:00',
               '2021-06-15 07:55:00+00:00', '2021-06-15 07:56:00+00:00',
               '2021-06-15 07:57:00+00:00', '2021-06-15 07:58:00+00:00',
               '2021-06-15 07:59:00+00:00', '2021-06-15 08:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=332, freq=None)

**NOTE:** as a consequence of treating both sides of the session as open, the number of minutes associated with the session is 332, which is two higher than the session duration (330 minutes). This is because both the open and close are treated as trading minutes (+1) and both the break-start and break-open are treated as trading minutes (another +1). NB For calendars/sessions without a break, the difference is +1.

In [17]:
hkg_neither.session_minutes(session)

DatetimeIndex(['2021-06-15 01:31:00+00:00', '2021-06-15 01:32:00+00:00',
               '2021-06-15 01:33:00+00:00', '2021-06-15 01:34:00+00:00',
               '2021-06-15 01:35:00+00:00', '2021-06-15 01:36:00+00:00',
               '2021-06-15 01:37:00+00:00', '2021-06-15 01:38:00+00:00',
               '2021-06-15 01:39:00+00:00', '2021-06-15 01:40:00+00:00',
               ...
               '2021-06-15 07:50:00+00:00', '2021-06-15 07:51:00+00:00',
               '2021-06-15 07:52:00+00:00', '2021-06-15 07:53:00+00:00',
               '2021-06-15 07:54:00+00:00', '2021-06-15 07:55:00+00:00',
               '2021-06-15 07:56:00+00:00', '2021-06-15 07:57:00+00:00',
               '2021-06-15 07:58:00+00:00', '2021-06-15 07:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=328, freq=None)

**NOTE** for the "neither" side the situation is reversed and the number of minutes associated with the session is two less than the session duration (or -1 for calendars/sessions that do not have a break).

These discrepancies will make themselves known in all methods that evaluate or interrogate minutes. **If you are not aware of how trading minutes are evaluated, you might get output that you weren't expecting...**

In [18]:
calendars =  [hkg_left, hkg_right, hkg_both, hkg_neither]
[ cal.sessions_minutes_count(session, session) for cal in calendars ]

[330, 330, 332, 328]

**NOTE**: the default side is **"left"**...

### Has it always been this way?

No.

The original `trading_calendars` package did not have a side option and treated trading minutes as if side were "right". This behaviour came about as a result of defining the open time as one minute later than the true open.

When support for `trading_calendars` ended `exchange_calendars` came into being. With the initial release 3.0 the opportunity was taken to amend open times to reflect the true open times. Trading minutes continued to be evaluated as previously which had the effect of treating trading minutes as if side were "both".

In release 3.4 the `ExchangeCalendar` side option was implemented to provide users with flexibility over how to treat trading minutes. In order to best preserve behaviour since release 3.0, the default side was "right" for the few 24-hour calendars and "both" for all others.

In release 4.0 the default side was set to "left" for all calendars. This is not expected to change going forwards.

### Can a `minute` parameter have a 'second' component?

Depends on the `side`...

#### 'left' and 'right'

Only if a calenar's side is either "left" or "right" can `minute` be passed with a second (or milisecond...) component. This is because only these sides allow for a minute timestamp to represent a specific sixty seconds...

Consider the minute timestamp **"2021-06-12 15:30:00"**:
* If the side is "left" then this timestamp will sit on the left side of the sixty seconds that it represents, that's to say it will represent the period from **"2021-06-12 15:30:00" through "2021-06-12 15:30:59"**. Indeed it will represent the period through to the instance before "2021-06-12 15:31:00", but it will NOT include "2021-06-12 15:31:00".
* If the side is "right" then the timestamp will sit on the right of the sixty seconds it represents, that's to say it will represent the period from the instance after "2021-06-12 15:30:00" through "2021-06-12 15:31:00". It will NOT include "2021-06-12 15:30:00" but will include "2021-06-12 15:31:00".

Thought of in this way it becomes clear why an exchange that closes at 16.00 <!--TODO - make sure using a time that follows the example above)--> is considered open at "16:00" when side is "right" but closed when the side is "left"...
* when the side is "right" the timestamp "16:00" refers to the sixty seconds from (but not including) 15:59 through 16:00, i.e. the last minute of the trading day during which the exchange was very much open.
* when the side is "left"  the timestamp "16:00" refers to the sixty seconds from 16:00 to (but not including) 16:01, i.e the first minute after the close when the exchange was very much closed.

When a `minute` parameter is receieved with a time component more specific than a minute then the corresponding minute timestamp is evaluated by simply rounding up or down if the `side` is "right" or "left" respectively.

In [19]:
hkg_left.next_minute("2021-06-15 05:25:30")

Timestamp('2021-06-15 05:26:00+0000', tz='UTC')

In [20]:
hkg_right.next_minute("2021-06-15 05:25:30")
# passed argument is represented by the trading minute "05:26", hence next_minute is...

Timestamp('2021-06-15 05:27:00+0000', tz='UTC')

In [21]:
hkg_left.is_open_on_minute("2021-06-15 07:59:59")

True

#### 'both' and 'neither'

If the calendar's `side` is either "both" or "neither" then the sixty second period that a minute timestamp refers to is ambiguous. Looked at the other way around, it's ambiguous which minute timestamp represents any specific second. In order to avoid this ambiguity, if the calendar's side is "both" or "neither" then any `minute` parameter must be received with a time component no more specific than a minute. To the contrary, a ValueError is raised...

In [ ]:
hkg_both.is_open_on_minute("2021-06-15 07:59:59")

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [22], in <cell line: 1>()
----> 1 hkg_both.is_open_on_minute("2021-06-15 07:59:59")

ValueError: `timestamp` cannot have a non-zero second (or more accurate) component for `side` 'both'. `timestamp` parsed as '2021-06-15 07:59:59+00:00'.
```

### `minute` timezone

`exchange_calendars` will always return minutes in terms of UTC. However, a `minute` parameter can take a timestamp of any timezone (`exchange_calendars` just parses it to UTC). Consider a session of the New York Stock Exchange:

In [23]:
nys = xcals.get_calendar("XNYS", side="left")
nys.schedule.loc[["2021-06-15"]]

,open,break_start,break_end,close
2021-06-15,2021-06-15 13:30:00+00:00,NaT,NaT,2021-06-15 20:00:00+00:00


In [24]:
nys.is_open_on_minute(pd.Timestamp("2021-06-15 09:35"))
# NB tz-naive input is assumed as UTC

False

In [25]:
nys.is_open_on_minute(pd.Timestamp("2021-06-15 09:35", tz=nys.tz))

True

In [26]:
nys.next_minute(pd.Timestamp("2021-06-15 09:35", tz=nys.tz))

Timestamp('2021-06-15 13:36:00+0000', tz='UTC')

In [27]:
nys.next_minute(pd.Timestamp("2021-06-15 09:35", tz=nys.tz)).tz_convert(nys.tz)

Timestamp('2021-06-15 09:36:00-0400', tz='America/New_York')

### `minute` bounds

Nearly all methods that have a `minute` parameter will require that `minute` is received as an object representing a minute that's no earlier than the calendar's first trading minute and no later than the calendar's last trading minute (this is always the case for a `minute` annotated with `TradingMinute`).

In [ ]:
nys.is_open_on_minute(nys.first_minute - pd.Timedelta(1, "T"))

```
---------------------------------------------------------------------------
MinuteOutOfBounds                         Traceback (most recent call last)
Input In [28], in <cell line: 1>()
----> 1 nys.is_open_on_minute(nys.first_minute - pd.Timedelta(1, "T"))

MinuteOutOfBounds: Parameter `minute` receieved as '2002-06-10 13:29:00+00:00' although cannot be earlier than the first trading minute of calendar 'XNYS' ('2002-06-10 13:30:00+00:00').
```

In [ ]:
nys.next_open(nys.last_minute + pd.Timedelta(1, "T"))

```
---------------------------------------------------------------------------
MinuteOutOfBounds                         Traceback (most recent call last)
Input In [29], in <cell line: 1>()
----> 1 nys.next_open(nys.last_minute + pd.Timedelta(1, "T"))

MinuteOutOfBounds: Parameter `minute` receieved as '2023-06-09 20:00:00+00:00' although cannot be later than the last trading minute of calendar 'XNYS' ('2023-06-09 19:59:00+00:00').
```